In [1]:
import os
from dotenv import load_dotenv
import pandas as pd
import sqlite3

from langchain.chains import SequentialChain, LLMChain
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory


pd.set_option('display.max_colwidth', None)  # No truncation of column content


# Initialize LLM

In [2]:
# Load environment variables from .env file
load_dotenv()

# Get the OPENAI_API_KEY from environment variables
openai_api_key = os.getenv('OPENAI_API_KEY')

llm = ChatOpenAI(api_key=openai_api_key, temperature=0, model="gpt-4o-mini")

# Initialize Database Connection for UDS Code Lookup

I found that OpenAI's 4o model had a surprisingly poor understanding of UDS codes. It couldn't correctly say which codes are UDS codes, nor the correct meaning of a given code.

I created an extract of UDS codes and their meanings, downloaded from [this website](https://uds.readthedocs.io/en/stable/pages/knowledge_base/diagnostic_message.html), broken down by service request (SID) codes and response codes (SID or non-response codes [NRC]). This dataset is stored under [./lookup/uds_codes.xlsx](./lookup/uds_codes.xlsx).

Some of the code explanations are verbose or poorly worded. I therefore use `gpt-4o-mini` to summarize these explanations before exporting to the lookup database. These summarized explanations are contained in the database column `Summary`.

In [3]:
sqlite_path = './lookup/uds_codes.db'  # Path to SQLite database
conn = sqlite3.connect(sqlite_path)

### Excerpt from UDS lookup database

Note - You'll notice below there are duplicate UDS codes. This is by design, as some SID codes have the same value as NRC codes. The primary keys for this table are [Code, Type]

In [4]:
codes = pd.read_sql_query(f"SELECT * FROM uds_codes;", conn)
print(codes['Code'].nunique())
print(codes.shape)
codes.head(10)

97
(121, 3)


,Code,Type,Summary
0,0x00,Negative Response Code,Negative Response Code: Positive response parameter reserved for server internal implementation.
1,0x10,Service Request,"Service Request: Controls diagnostic sessions on servers, enabling different services and functionalities."
2,0x10,Negative Response Code,"Negative Response Code: The server has rejected the requested action, indicated by a General Reject NRC."
3,0x11,Service Request,Service Request: ECU Reset: Client requests a server reset.
4,0x11,Negative Response Code,Negative Response Code: Service not supported; the server cannot fulfill the requested action.
5,0x12,Negative Response Code,Negative Response Code: Server cannot process the request due to unsupported service parameters.
6,0x13,Negative Response Code,Negative Response Code: NRC indicates request message length or format is incorrect for the specified service.
7,0x14,Service Request,Service Request: Clears all diagnostic information in one or multiple servers' memory.
8,0x14,Negative Response Code,Negative Response Code: Response exceeds maximum size allowed by network layer or server buffer limits.
9,0x19,Service Request,Service Request: Reads current Diagnostic Trouble Codes and related information from vehicle servers.


In [5]:
codes['Type'].value_counts()

Type
Negative Response Code    64
Service Response          29
Service Request           28
Name: count, dtype: int64

# Functions

In [6]:
# Define query functions for Request (SID) codes
def query_request_code(sid):
    """Query SQLite database for a SID code with 'Service Request' type."""
    query = "SELECT Summary FROM uds_codes WHERE Code = ? AND Type = 'Service Request'"
    result = pd.read_sql_query(query, conn, params=(sid,))
    return result.iloc[0, 0] if not result.empty else "Code not found"


# Define query function for Response (SID/NRC) codes
def query_response_code(reply):
    """Query SQLite database for a Reply code with either 'Service Response' or 'Negative Response Code' types."""
    query = "SELECT Summary FROM uds_codes WHERE Code = ? AND Type IN ('Service Response', 'Negative Response Code')"
    result = pd.read_sql_query(query, conn, params=(reply,))
    return result.iloc[0, 0] if not result.empty else "Code not found"

# Function for the lookup agent to interpret the Excel workbook
def lookup_codes_from_workbook(workbook_path) -> pd.Series:
    """Read an Excel workbook, look up explanations for each SID Request and SID/NRC Reply code, and return summaries as pd.Series."""
    
    # Load the workbook
    df = pd.read_excel(workbook_path)
    
    template = """ You are a helpful assistant that summarizes Unified Diagnostic Service (UDS) network communication between a SID request and SID/NRC response.
    
    Use the provided metadata to summarize what is being requested and the system response. Here is the data:
    
    Request: {request}
    Response: {response}
    
    Provide a concise (max. 15 word) summary using only the information given.
    
    """
    
    prompt = PromptTemplate(
        input_variables=["request", "response"],
        template=template,
    )
    
    chain = prompt | llm
    
    print("Creating summaries of each request-response message pair... \n")
    total_count = len(df)
    count_width = len(str(total_count))
    
    # Iterate through each row, looking up the request (SID) and response (SID/NRC) explanations and creating a summary
    summaries = []
    for count, row in df.iterrows():
        request_code = row['sid']
        response_code = row['reply']
        
        # Look up explanations
        request = query_request_code(request_code)
        response = query_response_code(response_code)
        
        summary = chain.invoke({"request": request, "response": response}).content
        print(f"{count+1:>{count_width}}", "    ", summary)
        
        # Append summary for each row
        summaries.append(summary)

    return pd.Series(summaries)


def summarize_messages(summaries: pd.Series) -> str:
    """ Summarize a series of text messages into a concise summary using the provided summaries."""
    
    template = """ Summarize the following messages in the provided series of text messages.
    
    Messages: {messages}
    
    Provide a concise (max. 25 word) summary using only the information given.
    
    """
    
    prompt = PromptTemplate(
        input_variables=["messages"],
        template=template,
    )
    
    chain = prompt | llm
    
    result = chain.invoke({"messages": summaries.to_string()}).content
    
    print(result)
    
    return result

# Diagnostic Files

In [7]:
files = [i for i in os.listdir('./data') if i.endswith('.xlsx')]
files

['PreconditionCheckFailA2.xlsx',
 'EMSSecurityAccessFailure.xlsx',
 'EMSSessionNoRes.xlsx',
 'Endless78.xlsx',
 'EMSSessionNoResp.xlsx',
 'EMSSession11.xlsx',
 'sessionfailureEMS.xlsx',
 'EMSSession13.xlsx',
 'vinfailure_stop.xlsx',
 'EMSSession12.xlsx',
 'PreconditionCheckNegResp.xlsx',
 'vinfailure_continue.xlsx',
 'EMSSession14.xlsx']

# Generate Summary

In [8]:
summaries = lookup_codes_from_workbook(os.path.join('./data', files[0]))

Creating summaries of each request-response message pair... 

 1      Client connects to vehicle; server responds with internal implementation code, no positive response.
 2      Client connects to vehicle; response indicates code not found.
 3      Client connects to vehicle; server responds with internal implementation code, no positive response.
 4      Client connected to vehicle; active diagnostic services requested, but code not found.
 5      Client requests data records; server responds with a negative response code.
 6      Client requests data records; server responds with "Code not found."
 7      Request to control diagnostic sessions; response indicates reserved positive parameter for internal use.
 8      Request for diagnostic session control; response indicates the code was not found.
 9      Client connects to vehicle; server responds with internal implementation code, no positive response.
10      Client connected to vehicle; diagnostic service request resulted in "Co

In [9]:
cumulative_summary = summarize_messages(summaries)

Client repeatedly connects to the vehicle, but encounters multiple "code not found" responses and reserved parameters for internal use across various requests.
